In [ ]:
#@title <center><img src='https://i.imgur.com/Vz0uo7C.png' width='275'>
OMDB_API         = "" #@param {type: 'string'} 
REMOTE_NAME      = "" #@param {type: 'string'} 
SHARED_DRIVE     = "" #@param {type: 'string'}
RCLONE_PASSWORD  = "" #@param {type: 'string'}
HIDE_POST_COUNT  = 25 #@param {type: 'integer'} 
HIDE_REACT_SCORE = 25 #@param {type: 'integer'} 
 
from google.colab import files
from IPython.display import clear_output
 
## Install FFmpeg, MediaInfo, rClone
!wget -q 'https://github.com/InspectorWilliamHenderson/rFFmedia/releases/latest/download/rFFmedia.zip' && unzip -o -qq './rFFmedia.zip' -d '/' && rm -rf '/content/sample_data' './rFFmedia.zip'
 
## Configure rClone
!echo 'Upload the rClone.conf'
uploaded = files.upload()
clear_output()
!rclone --config '' move './rclone.conf' '/root/.config/rclone/'
 
## rClone Mount
if OMDB_API == '':
  OMDB_API = 'c42fc7df'
if SHARED_DRIVE != '':
  SHARED_DRIVE = f'--drive-root-folder-id={SHARED_DRIVE}'
if RCLONE_PASSWORD != '':
  %env RCLONE_CONFIG_PASS=$RCLONE_PASSWORD
  clear_output()
!mkdir '/content/{REMOTE_NAME}/'
!rclone mount '{REMOTE_NAME}': '/content/{REMOTE_NAME}/' --buffer-size 32M --daemon $SHARED_DRIVE --max-read-ahead 128M --poll-interval 30s --read-only --use-mmap --vfs-cache-mode full --vfs-read-chunk-size-limit 32M --vfs-read-chunk-size 16M -v

In [ ]:
#@title <h3><b><center>Movie and TV Shows Template
import base64, os, re, requests, subprocess, threading
 
def GetMediaInfo(PATH):
  global MEDIAINFO
  MEDIAINFO = subprocess.check_output(['mediainfo', PATH]).decode().strip()
  MEDIAINFO = MEDIAINFO.replace(PATH, os.path.basename(PATH))
 
def CreateScreenshots(PATH, TIME):
  global SCREENSHOTS
  LINK = subprocess.Popen(['ffmpeg', '-ss', TIME, '-i', PATH, '-frames:v', '1', '-y', f'./frame_{TIME}.jpeg'], stdin=open(os.devnull, 'wb'), stdout=open(os.devnull, 'wb')).communicate()
  if os.path.exists(f'./frame_{TIME}.jpeg'):
    LINK = requests.post('https://api.imgbb.com/1/upload', {'key': '6b1c29d7993f76b154e2c704f1d13879', 'image': base64.b64encode(open(f'./frame_{TIME}.jpeg', 'rb').read())})
    if str(LINK.status_code) == '200':
      SCREENSHOTS += f"[URL='{LINK.json()['data']['url']}'][IMG]{LINK.json()['data']['url']}[/IMG][/URL]\n"
 
def InitiateScreenshots():
  global SCREENSHOTS
  if ('.DoVi.' in os.path.basename(PATH)) or ('.DV.' in os.path.basename(PATH)):
    SCREENSHOTS = "[HR][/HR][INDENT][SIZE=6][FORUMCOLOR][B]Media Screenshots :[/B][/FORUMCOLOR][/SIZE][/INDENT]\n[SPOILER='Media Screenshots'][JUSTIFY]Screenshots Can't be Made for This Release Because There is No Tool to Make Screenshots Out of a Muxed Dolby Vision File. There is a Very Limited Amount of Devices that can Play Dolby Vision Muxed this Way so Keep that in Mind Before Downloading.​[/JUSTIFY][/SPOILER]"
  elif SCREENSHOTS == True:
    SCREENSHOTS = "[HR][/HR][INDENT][SIZE=6][FORUMCOLOR][B]Media Screenshots :[/B][/FORUMCOLOR][/SIZE][/INDENT]\n[SPOILER='Media Screenshots'][CENTER]"
    SCREENSHOTS_01 = threading.Thread(target=CreateScreenshots(PATH, '02:00.00'))
    SCREENSHOTS_02 = threading.Thread(target=CreateScreenshots(PATH, '04:00.00'))
    SCREENSHOTS_03 = threading.Thread(target=CreateScreenshots(PATH, '06:00.00'))
    SCREENSHOTS_01.start()
    SCREENSHOTS_02.start()
    SCREENSHOTS_03.start()
    SCREENSHOTS_01.join()
    SCREENSHOTS_02.join()
    SCREENSHOTS_03.join()
    SCREENSHOTS += '[/CENTER][/SPOILER]'
    if SCREENSHOTS == "[HR][/HR][INDENT][SIZE=6][FORUMCOLOR][B]Media Screenshots :[/B][/FORUMCOLOR][/SIZE][/INDENT]\n[SPOILER='Media Screenshots'][CENTER][/CENTER][/SPOILER]":
      SCREENSHOTS = ''
  else:
    SCREENSHOTS = ''
 
def GenerateLink(PATH, FOLDER):
  global LINK
  if LINK == "":
    LINK = PATH.replace(f'/content/{REMOTE_NAME}/', '')
    if FOLDER != 'Disabled':
      LINK = os.path.dirname(LINK)
      if FOLDER == 'GRANDPARENT FOLDER':
        LINK = os.path.dirname(LINK)
    if SHARED_DRIVE == '':
      LINK = subprocess.check_output(['rclone', 'link', f'{REMOTE_NAME}:{LINK}']).decode().strip()
    else:
      LINK = subprocess.check_output(['rclone', 'link', f'{REMOTE_NAME}:{LINK}', SHARED_DRIVE]).decode().strip()
    LINK = ''.join(LINK)
  LINK = f"[HIDEREACT=1,2,3,4,5,6,7,8][DOWNCLOUD]{LINK}[/DOWNCLOUD][/HIDEREACT]"
  if HIDE_REACT_SCORE != 0:
    LINK = f'[HIDEREACTSCORE={HIDE_REACT_SCORE}]{LINK}[/HIDEREACTSCORE]'
  if HIDE_POST_COUNT != 0:
    LINK = f'[HIDEPOSTS={HIDE_POST_COUNT}]{LINK}[/HIDEPOSTS]'
 
def GetIMDb(PATH):
  global IMDB
  if IMDB == '':
    try:
      YEAR = '&y=' + re.findall(r'(19\d{2}|20\d{2})', PATH)[-1]
    except:
      YEAR = ''
    TITLE = re.sub('[^a-zA-Z0-9]', '+', os.path.basename(PATH)).split('+' + YEAR.replace('&y=', '') + '+')[0].split('+S0')[0].split('+S1')[0].split('+S2')[0].split('+1080p')[0].split('+2160p')[0].split('+UHD')[0].split('+REPACK')[0].split('+HYBRID')[0].split('+EXTENDED')[0]
    IMDB = requests.get(f'http://www.omdbapi.com/?t={TITLE}{YEAR}&apikey={OMDB_API}&r=json').json()
  else:
    IMDB = re.search(r'tt([0-9]+)', IMDB)[1].lstrip('0').zfill(7)
    IMDB = requests.get(f'http://www.omdbapi.com/?i=tt{IMDB}&apikey={OMDB_API}&r=json').json()
 
def GetYouTube(QUERY):
  global YOUTUBE
  try:
    if YOUTUBE == '':
      YOUTUBE = 'https://www.youtube.com/watch?v=' + re.search(r'watch\?v=(\S{11})', requests.get(f'https://www.youtube.com/results?search_query={QUERY}+Official+Trailer').text)[1]
    else:
      YOUTUBE = 'https://www.youtube.com/watch?v=' + re.findall(r'([A-Za-z0-9-_]{11})', YOUTUBE)[-1]
  except:
    YOUTUBE = ''
 
def BlackPearl_Posting_Template():
  global FOLDER, IMDB, LINK, PATH, SCREENSHOTS, YOUTUBE
  !rm -f ./frame*.jpeg
  GetMediaInfo_THREAD = threading.Thread(target=GetMediaInfo(PATH))
  InitiateScreenshots_THREAD = threading.Thread(target=InitiateScreenshots())
  GenerateLink_THREAD = threading.Thread(target=GenerateLink(PATH, FOLDER))
  GetIMDb_THREAD = threading.Thread(target=GetIMDb(PATH))
  GetMediaInfo_THREAD.start()
  InitiateScreenshots_THREAD.start()
  GenerateLink_THREAD.start()
  GetIMDb_THREAD.start()
  GetIMDb_THREAD.join()
  if IMDB['Response'] == 'False':
      print('Give IMDb ID')
      return None
  GetYouTube_THREAD = threading.Thread(target=GetYouTube((IMDB['Title'] + " " + IMDB['Year'][0:4]).replace(' ', '+')))
  GetYouTube_THREAD.start()
  BBCODE = ''
  if ('Poster' in IMDB and IMDB['Poster'] != 'N/A'):
    IMDB['Poster'] = re.sub('_V1_SX\d+.jpg', '_V1_SX1000.png', IMDB['Poster'])
    BBCODE = f"[CENTER][URL='{IMDB['Poster']}'][IMG WIDTH='350px']{IMDB['Poster']}[/IMG][/URL][/CENTER]\n"
  else:
    BBCODE = f"[CENTER][SIZE=160px][B]\n\n\n\n\nPOSTER\nMISSING\n\n\n\n\n\n[/B][/SIZE][/CENTER]\n"
  BBCODE += f"[CENTER][URL='https://blackpearl.biz/search/1/?q={IMDB['imdbID']}&o=date'][FORUMCOLOR][B][SIZE=6]{IMDB['Title']} ({IMDB['Year'][0:4]})[/SIZE][/B][/FORUMCOLOR][/URL][/CENTER]\n"
  BBCODE += f"[CENTER][URL='https://imdb.com/title/{IMDB['imdbID']}'][IMG WIDTH='46px']https://ia.media-imdb.com/images/M/MV5BMTk3ODA4Mjc0NF5BMl5BcG5nXkFtZTgwNDc1MzQ2OTE@.png[/IMG][/URL][/CENTER]"
  BBCODE += f"[HR][/HR][INDENT][SIZE=6][FORUMCOLOR][B]Plot Summary :[/B][/FORUMCOLOR][/SIZE]\n\n[JUSTIFY]{IMDB['Plot']}[/JUSTIFY][/INDENT]" if ('Plot' in IMDB and IMDB['Plot'] != 'N/A') else ''
  if ('Type' in IMDB and IMDB['Type'] == 'movie'):
    IMDB['Type'] = 'Movie'
  elif ('Type' in IMDB and IMDB['Type'] == 'series'):
    IMDB['Type'] = 'TV Show'
  else:
    IMDB['Type'] = 'IMDb'
  BBCODE += f"[HR][/HR][INDENT][SIZE=6][FORUMCOLOR][B]{IMDB['Type']} Info :[/B][/FORUMCOLOR][/SIZE][/INDENT]\n[LIST]"
  BBCODE += f"[*][FORUMCOLOR][B]IMDb :[/B][/FORUMCOLOR] {IMDB['imdbRating']} ({IMDB['imdbVotes']})\n" if ('imdbRating' in IMDB and IMDB['imdbRating'] != 'N/A' and 'imdbVotes' in IMDB and IMDB['imdbVotes'] != 'N/A') else ''
  BBCODE += f"[*][FORUMCOLOR][B]Rated :[/B][/FORUMCOLOR] {IMDB['Rated']}\n" if ('Rated' in IMDB and IMDB['Rated'] != 'N/A') else ''
  BBCODE += f"[*][FORUMCOLOR][B]Genres :[/B][/FORUMCOLOR] {IMDB['Genre']}\n" if ('Genre' in IMDB and IMDB['Genre'] != 'N/A') else ''
  BBCODE += f"[*][FORUMCOLOR][B]Awards :[/B][/FORUMCOLOR] {IMDB['Awards']}\n" if ('Awards' in IMDB and IMDB['Awards'] != 'N/A') else ''
  BBCODE += f"[*][FORUMCOLOR][B]Runtime :[/B][/FORUMCOLOR] {IMDB['Runtime']}\n" if ('Runtime' in IMDB and IMDB['Runtime'] != 'N/A') else ''
  BBCODE += f"[*][FORUMCOLOR][B]Starring :[/B][/FORUMCOLOR] {IMDB['Actors']}\n" if ('Actors' in IMDB and IMDB['Actors'] != 'N/A') else ''
  BBCODE += f"[*][FORUMCOLOR][B]Countries :[/B][/FORUMCOLOR] {IMDB['Country']}\n" if ('Country' in IMDB and IMDB['Country'] != 'N/A') else ''
  BBCODE += f"[*][FORUMCOLOR][B]Languages :[/B][/FORUMCOLOR] {IMDB['Language']}\n" if ('Language' in IMDB and IMDB['Language'] != 'N/A') else ''
  BBCODE += f"[*][FORUMCOLOR][B]Written By :[/B][/FORUMCOLOR] {IMDB['Writer']}\n" if ('Writer' in IMDB and IMDB['Writer'] != 'N/A') else ''
  BBCODE += f"[*][FORUMCOLOR][B]Directed By :[/B][/FORUMCOLOR] {IMDB['Director']}\n" if ('Director' in IMDB and IMDB['Director'] != 'N/A') else ''
  BBCODE += f"[*][FORUMCOLOR][B]Release Date :[/B][/FORUMCOLOR] {IMDB['Released']}\n" if ('Released' in IMDB and IMDB['Released'] != 'N/A') else ''
  BBCODE += f"[*][FORUMCOLOR][B]Production By :[/B][/FORUMCOLOR] {IMDB['Production']}\n" if ('Production' in IMDB and IMDB['Production'] != 'N/A') else ''
  BBCODE += f"[*][FORUMCOLOR][B]DVD Release Date:[/B][/FORUMCOLOR] {IMDB['DVD']}\n" if ('DVD' in IMDB and IMDB['DVD'] != 'N/A') else ''
  BBCODE += f"[*][FORUMCOLOR][B]Official Website :[/B][/FORUMCOLOR] {IMDB['Website']}\n" if ('Website' in IMDB and IMDB['Website'] != 'N/A') else ''
  BBCODE += f"[*][FORUMCOLOR][B]Box Office Collection :[/B][/FORUMCOLOR] {IMDB['BoxOffice']}[/LIST]" if ('BoxOffice' in IMDB and IMDB['BoxOffice'] != 'N/A') else '[/LIST]'
  BBCODE = BBCODE.replace('[HR][/HR][INDENT][SIZE=6][FORUMCOLOR][B]IMDb Info :[/B][/FORUMCOLOR][/SIZE][/INDENT]\n[LIST][/LIST]', '')
  GetYouTube_THREAD.join()
  BBCODE += f'[HR][/HR][INDENT][SIZE=6][FORUMCOLOR][B]Official Trailer :[/B][/FORUMCOLOR][/SIZE][/INDENT]\n\n[CENTER]{YOUTUBE}[/CENTER]' if YOUTUBE != '' else ''
  BBCODE += '[HR][/HR][INDENT][SIZE=6][FORUMCOLOR][B]Media Info :[/B][/FORUMCOLOR][/SIZE][/INDENT]'
  GetMediaInfo_THREAD.join()
  InitiateScreenshots_THREAD.join()
  BBCODE += f"[SPOILER='Media Info'][CODE TITLE='Media Info']{MEDIAINFO}[/CODE][/SPOILER]\n{SCREENSHOTS}"
  GenerateLink_THREAD.join()
  BBCODE += f'[HR][/HR][INDENT][SIZE=6][FORUMCOLOR][B]Download Link :[/B][/FORUMCOLOR][/SIZE][/INDENT]\n[CENTER]{LINK}[/CENTER]'
  print(BBCODE)
  !rm -f ./frame*.jpeg

In [ ]:
IMDB        = "" #@param {type: 'string'}
YOUTUBE     = "" #@param {type: 'string'}
PATH        = "" #@param {type: 'string'}
LINK        = "" #@param {type: 'string'}
FOLDER      = "Disabled" #@param ['Disabled', 'PARENT FOLDER', 'GRANDPARENT FOLDER']
SCREENSHOTS = False #@param {type: 'boolean'}
 
try:
  BlackPearl_Posting_Template()
except NameError:
  print("Run Second Cell Before Running The Third Cell")
 
if FOLDER != 'Disabled':
  PATH = os.path.dirname(PATH)
  if FOLDER == 'GRANDPARENT FOLDER':
    PATH = os.path.dirname(PATH)
  PATH = PATH.replace(f'/content/{REMOTE_NAME}/', '')
  !rclone size '{REMOTE_NAME}':'{PATH}'